In [ ]:
# Install required packages
!pip install fastapi uvicorn python-multipart requests pillow deepface pyngrok nest-asyncio motor numpy opencv-python

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import requests
from io import BytesIO
from deepface import DeepFace
from pyngrok import ngrok, conf
import nest_asyncio
import threading
import os
from google.colab import userdata
from motor.motor_asyncio import AsyncIOMotorClient
from datetime import datetime
import asyncio
import time
import numpy as np
import cv2
import json

# Allow nested event loop for Colab
nest_asyncio.apply()

# Load secrets from Google Colab
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    MONGODB_URI = userdata.get('MONGODB_URI')
    print("Secrets loaded successfully")
except Exception as e:
    print(f"Error loading secrets: {e}")
    print("Make sure to add NGROK_TOKEN and MONGODB_URI in Colab secrets")

# Set up ngrok authentication
conf.get_default().auth_token = NGROK_TOKEN

# Initialize MongoDB client
mongo_client = AsyncIOMotorClient(MONGODB_URI)
db = mongo_client['face_recognition']
uploads_collection = db['uploads']

app = FastAPI()

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ImageRequest(BaseModel):
    image_url: str

def aggressive_python_convert(val):
    """EXTREMELY aggressive conversion to Python native types"""
    # Handle None
    if val is None:
        return None

    # Handle already Python types
    if isinstance(val, (int, float, str, bool)):
        return val

    # Handle numpy arrays
    if isinstance(val, np.ndarray):
        return val.tolist()

    # Handle dictionaries recursively
    if isinstance(val, dict):
        return {str(k): aggressive_python_convert(v) for k, v in val.items()}

    # Handle lists/tuples recursively
    if isinstance(val, (list, tuple)):
        return [aggressive_python_convert(item) for item in val]

    # Convert ANY numpy numeric type to Python equivalent
    val_type_str = str(type(val))
    if 'numpy' in val_type_str:
        try:
            if hasattr(val, 'item'):
                return aggressive_python_convert(val.item())
            elif 'int' in val_type_str:
                return int(val)
            elif 'float' in val_type_str:
                return float(val)
            elif 'bool' in val_type_str:
                return bool(val)
            else:
                return float(val)  # Default to float for unknown numpy types
        except:
            return str(val)  # Last resort

    # For any other type, try to convert to basic Python type
    try:
        if hasattr(val, 'item'):
            return aggressive_python_convert(val.item())
        else:
            return str(val)
    except:
        return str(val)

def extract_filename_from_url(url):
    """Extract filename from URL"""
    try:
        filename = url.split('/')[-1].split('?')[0]
        return filename
    except:
        return "unknown-image.jpg"

@app.post("/recognize")
async def recognize(request: ImageRequest):
    try:
        print(f"Processing image: {request.image_url}")

        # Download image from URL
        response = requests.get(request.image_url, timeout=30)
        response.raise_for_status()

        # Use OpenCV to decode image directly from bytes
        img_array = np.frombuffer(response.content, np.uint8)
        img_bgr = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        if img_bgr is None:
            raise ValueError("Could not decode image")
        print(f"Image shape: {img_bgr.shape}, dtype: {img_bgr.dtype}")

        # Use BGR numpy array with DeepFace
        analysis = DeepFace.analyze(
            img_bgr,
            actions=['age', 'gender', 'emotion'],
            enforce_detection=False
        )
        print(f"Raw analysis result type: {type(analysis)}")

        # Extract data
        if isinstance(analysis, list) and len(analysis) > 0:
            data = analysis[0]
        elif isinstance(analysis, dict):
            data = analysis
        else:
            return {"success": False, "error": "No faces detected"}

        # BRUTALLY extract and convert values
        raw_age = data.get('age', 0)
        raw_gender = data.get('dominant_gender', '')
        raw_emotion = data.get('dominant_emotion', '')
        raw_emotion_dict = data.get('emotion', {})

        # Get emotion confidence and FORCE convert immediately
        raw_emotion_confidence = 0.0
        if raw_emotion and raw_emotion in raw_emotion_dict:
            raw_emotion_confidence = raw_emotion_dict[raw_emotion]

        # IMMEDIATELY convert each value aggressively
        age = int(aggressive_python_convert(raw_age))
        gender = str(aggressive_python_convert(raw_gender))
        emotion = str(aggressive_python_convert(raw_emotion))
        emotion_confidence = float(aggressive_python_convert(raw_emotion_confidence))
        processed_at = str(datetime.utcnow().isoformat())

        print(f"Converted values:")
        print(f"  age: {age} (type: {type(age)})")
        print(f"  gender: {gender} (type: {type(gender)})")
        print(f"  emotion: {emotion} (type: {type(emotion)})")
        print(f"  emotion_confidence: {emotion_confidence} (type: {type(emotion_confidence)})")

        # Create result with 100% Python native types
        result = {
            'age': age,
            'gender': gender,
            'emotion': emotion,
            'emotion_confidence': emotion_confidence,
            'processedAt': processed_at
        }

        # FINAL aggressive conversion of entire result
        result = aggressive_python_convert(result)

        print(f"Final result: {result}")
        print(f"Final result types: {[(k, type(v).__name__) for k, v in result.items()]}")

        # Test JSON serialization
        try:
            json.dumps(result)
            print("✅ JSON serialization test PASSED")
        except Exception as json_error:
            print(f"❌ JSON serialization FAILED: {json_error}")
            return {"success": False, "error": f"Type conversion failed: {json_error}"}

        # MongoDB upsert with aggressive retry
        max_attempts = 2
        mongodb_success = False

        for attempt in range(max_attempts):
            try:
                print(f"📈 MongoDB attempt {attempt + 1}/{max_attempts}")

                # Create fresh connection
                mongo_client_robust = AsyncIOMotorClient(MONGODB_URI)
                db_robust = mongo_client_robust['face_recognition']

                # Create upsert payload
                filename = extract_filename_from_url(request.image_url)

                update_payload = {
                    "$set": {
                        "recognitionResults": result
                    },
                    "$setOnInsert": {
                        "imageUrl": request.image_url,
                        "filename": filename,
                        "uploadedAt": datetime.utcnow(),
                        "userFeedback": None
                    }
                }

                # Execute upsert
                update_result = await db_robust['uploads'].update_one(
                    {"imageUrl": request.image_url},
                    update_payload,
                    upsert=True
                )

                await mongo_client_robust.close()

                if update_result.modified_count > 0 or update_result.upserted_id is not None:
                    action = "created" if update_result.upserted_id else "updated"
                    print(f"✅ MongoDB {action} successfully on attempt {attempt + 1}")
                    mongodb_success = True
                    break
                else:
                    print(f"⚠️ No changes made on attempt {attempt + 1}")

            except Exception as mongo_error:
                print(f"❌ MongoDB attempt {attempt + 1} failed: {mongo_error}")
                try:
                    await mongo_client_robust.close()
                except:
                    pass

                if attempt < max_attempts - 1:
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff

        if mongodb_success:
            print(f"✅ MongoDB operation completed successfully")
        else:
            print(f"⚠️ MongoDB operation failed after {max_attempts} attempts")

        return {"success": True, "result": result}

    except Exception as e:
        print(f"💥 Error in recognize function: {e}")
        import traceback
        traceback.print_exc()
        return {"success": False, "error": str(e)}

@app.get("/")
async def health_check():
    return {
        "status": "Facial analytics API is running",
        "timestamp": datetime.utcnow().isoformat(),
        "mongodb_connected": True if mongo_client else False
    }

@app.get("/test-db")
async def test_database():
    try:
        result = await uploads_collection.count_documents({})
        return {"mongodb_status": "connected", "total_uploads": result}
    except Exception as e:
        return {"mongodb_status": "error", "error": str(e)}

# AGGRESSIVE server cleanup and restart
import subprocess
import psutil

def kill_existing_servers():
    """Kill any existing servers on port 8000"""
    try:
        # Kill by process name
        subprocess.run(["pkill", "-f", "uvicorn"], check=False, capture_output=True)
        time.sleep(1)

        # Kill by port
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                for conn in proc.info['connections'] or []:
                    if conn.laddr.port == 8000:
                        print(f"Killing process {proc.info['pid']} using port 8000")
                        proc.kill()
            except:
                pass

        time.sleep(2)
        print("✅ Existing servers cleaned up")
    except Exception as e:
        print(f"⚠️ Cleanup warning: {e}")

# Clean up existing servers
kill_existing_servers()

# Global variables
server_thread = None
ngrok_tunnel = None

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")  # Set to error to reduce noise

# Start server
print("🚀 Starting server...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

# Set up ngrok tunnel
try:
    # Disconnect any existing tunnels
    ngrok.kill()
    time.sleep(1)

    ngrok_tunnel = ngrok.connect(8000)
    print("\n" + "="*60)
    print("✅ FACIAL ANALYTICS API IS READY!")
    print("="*60)
    print(f"🌐 Public URL: {ngrok_tunnel.public_url}")
    print(f"🔗 API endpoint: {ngrok_tunnel.public_url}/recognize")
    print(f"💊 Health check: {ngrok_tunnel.public_url}/")
    print(f"🗄️ Database test: {ngrok_tunnel.public_url}/test-db")
    print("\n📝 Copy this to your .env.local:")
    print(f"NEXT_PUBLIC_COLAB_API_URL={ngrok_tunnel.public_url}")
    print("="*60)

    # Keep alive
    try:
        while True:
            time.sleep(300)  # Check every 5 minutes
            current_time = datetime.now().strftime('%H:%M:%S')
            print(f"⏰ Server running... {current_time}")
    except KeyboardInterrupt:
        print("🛑 Server stopped by user")
        ngrok.disconnect(ngrok_tunnel.public_url)

except Exception as e:
    print(f"💥 Error setting up ngrok: {e}")

Secrets loaded successfully
✅ Existing servers cleaned up
🚀 Starting server...

✅ FACIAL ANALYTICS API IS READY!
🌐 Public URL: https://605e-34-124-211-44.ngrok-free.app
🔗 API endpoint: https://605e-34-124-211-44.ngrok-free.app/recognize
💊 Health check: https://605e-34-124-211-44.ngrok-free.app/
🗄️ Database test: https://605e-34-124-211-44.ngrok-free.app/test-db

📝 Copy this to your .env.local:
NEXT_PUBLIC_COLAB_API_URL=https://605e-34-124-211-44.ngrok-free.app
Processing image: https://pub-ce7ee2cbf5c343bdaad85d47eb379e07.r2.dev//1750684028772-mirror.jpg
Image shape: (1069, 1080, 3), dtype: uint8


Action: emotion: 100%|██████████| 3/3 [00:04<00:00,  1.41s/it]


Raw analysis result type: <class 'list'>
Converted values:
  age: 25 (type: <class 'int'>)
  gender: Man (type: <class 'str'>)
  emotion: neutral (type: <class 'str'>)
  emotion_confidence: 59.240142822265625 (type: <class 'float'>)
Final result: {'age': 25, 'gender': 'Man', 'emotion': 'neutral', 'emotion_confidence': 59.240142822265625, 'processedAt': '2025-06-23T13:07:15.625492'}
Final result types: [('age', 'int'), ('gender', 'str'), ('emotion', 'str'), ('emotion_confidence', 'float'), ('processedAt', 'str')]
✅ JSON serialization test PASSED
📈 MongoDB attempt 1/5
❌ MongoDB attempt 1 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 2/5
❌ MongoDB attempt 2 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 3/5
❌ MongoDB attempt 3 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 4/5
❌ MongoDB attempt 4 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 5/5
❌ MongoDB attempt 5

Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 31.01it/s]


Raw analysis result type: <class 'list'>
Converted values:
  age: 30 (type: <class 'int'>)
  gender: Man (type: <class 'str'>)
  emotion: sad (type: <class 'str'>)
  emotion_confidence: 89.45646667480469 (type: <class 'float'>)
Final result: {'age': 30, 'gender': 'Man', 'emotion': 'sad', 'emotion_confidence': 89.45646667480469, 'processedAt': '2025-06-23T13:12:09.034734'}
Final result types: [('age', 'int'), ('gender', 'str'), ('emotion', 'str'), ('emotion_confidence', 'float'), ('processedAt', 'str')]
✅ JSON serialization test PASSED
📈 MongoDB attempt 1/5
❌ MongoDB attempt 1 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 2/5
❌ MongoDB attempt 2 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 3/5
❌ MongoDB attempt 3 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 4/5
❌ MongoDB attempt 4 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 5/5
❌ MongoDB attempt 5 failed: o

...

Raw analysis result type: <class 'list'>
Converted values:
  age: 31 (type: <class 'int'>)
  gender: Man (type: <class 'str'>)
  emotion: sad (type: <class 'str'>)
  emotion_confidence: 71.63528442382812 (type: <class 'float'>)
Final result: {'age': 31, 'gender': 'Man', 'emotion': 'sad', 'emotion_confidence': 71.63528442382812, 'processedAt': '2025-06-23T13:21:22.339965'}
Final result types: [('age', 'int'), ('gender', 'str'), ('emotion', 'str'), ('emotion_confidence', 'float'), ('processedAt', 'str')]
✅ JSON serialization test PASSED
📈 MongoDB attempt 1/5


❌ MongoDB attempt 1 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 2/5
❌ MongoDB attempt 2 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 3/5
❌ MongoDB attempt 3 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 4/5
❌ MongoDB attempt 4 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 5/5
❌ MongoDB attempt 5 failed: object NoneType can't be used in 'await' expression
⚠️ MongoDB operation failed after 5 attempts
⏰ Server running... 13:21:46
Processing image: https://pub-ce7ee2cbf5c343bdaad85d47eb379e07.r2.dev/1750685077680-messaging icon black.png
Image shape: (329, 401, 3), dtype: uint8


Action: emotion: 100%|██████████| 3/3 [00:00<00:00, 49.67it/s]

Raw analysis result type: <class 'list'>
Converted values:
  age: 44 (type: <class 'int'>)
  gender: Man (type: <class 'str'>)
  emotion: angry (type: <class 'str'>)
  emotion_confidence: 82.04037475585938 (type: <class 'float'>)
Final result: {'age': 44, 'gender': 'Man', 'emotion': 'angry', 'emotion_confidence': 82.04037475585938, 'processedAt': '2025-06-23T13:24:38.220250'}
Final result types: [('age', 'int'), ('gender', 'str'), ('emotion', 'str'), ('emotion_confidence', 'float'), ('processedAt', 'str')]
✅ JSON serialization test PASSED
📈 MongoDB attempt 1/5


❌ MongoDB attempt 1 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 2/5
❌ MongoDB attempt 2 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 3/5
❌ MongoDB attempt 3 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 4/5
❌ MongoDB attempt 4 failed: object NoneType can't be used in 'await' expression
📈 MongoDB attempt 5/5
❌ MongoDB attempt 5 failed: object NoneType can't be used in 'await' expression
⚠️ MongoDB operation failed after 5 attempts
⏰ Server running... 13:26:46
⏰ Server running... 13:31:46
⏰ Server running... 13:36:46
⏰ Server running... 13:41:46
⏰ Server running... 13:46:46
⏰ Server running... 13:51:46
⏰ Server running... 13:56:46
🛑 Server stopped by user
💥 Error setting up ngrok: [Errno 104] Connection reset by peer
